## softmax calculation

$$
x = [x_1, x_2, ..., x_B] \in R^B  \\

m(x) := \max_i \ x_i \\

f(x) := [e^{x_1 - m(x)} \ ... \  e^{x_B - m(x)}] \\

l(x) := \sum_i f(x)_i \\

softmax(x) := \frac {f(x)} {l(x)} 
$$



In [1]:
import numpy as np

In [ ]:

# 计算公式
# S = Q * (K.transpose(-1, -2))
# P = softmax(S) 
# O = P * V
import numpy as np

N = 4
d = 2

S = np.random.random(size=(1, N))
V = np.random.random(size=(N, d))

def tiled_softmax_then_matmul(S, V):
    acc = np.zeros(shape=(1, d))
    pre_max = float("-inf")
    pre_sum = 0
    for i  in range(N): # 每个token，KV的列维度，为了简洁，这里把Q的行维度设为了1，因此没有了内循环
        s_i = S[:,i] # 每列S
        cur_max = max(pre_max, s_i) # 当前分块和之前分块一起的最大值
        pre_sum *= (np.exp(pre_max - cur_max)) # L10
        cur_sum = pre_sum + np.exp(s_i - cur_max) # 当前分块和之前分块一起的指数和
        score = np.exp(s_i - cur_max) / cur_sum # 当前分块的softmax结果
        scale = pre_sum / cur_sum # 因为上一个分块的结果是基于当时的softmax中间sum组成的分母（presum），现在这个分块又得到了新的中间sum（cursum），所以需要更新：对上一个分块的结果acc做一个scale，保证结果的正确性
        acc *= scale 
        acc += score * V[i,] # scale后的中间结果加上当前分块的P * V = O
        pre_max = cur_max 
        pre_sum = cur_sum 
    return acc